## Imports and Constants

In [45]:
import pandas as pd
import json
import ipaddress

In [2]:
TRANSACTION_FRAUD_DATA_PATH = "./transaction_fraud_data.parquet"
HISTORICAL_CURRENCY_EXCHANGE_PATH = "./historical_currency_exchange.parquet"

## Download data

In [3]:
transaction_fraud_data = pd.read_parquet(TRANSACTION_FRAUD_DATA_PATH)
transaction_fraud_data.head(5)

,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,country,...,is_card_present,device,channel,device_fingerprint,ip_address,is_outside_home_country,is_high_risk_vendor,is_weekend,last_hour_activity,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,False,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,False,False,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,False,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,True,True,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,False,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,False,False,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466,Gas,major,Exxon,630.60,AUD,Australia,...,False,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,False,False,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,False,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,True,False,False,"{'num_transactions': 218, 'total_amount': 4827...",True


In [4]:
historical_currency_exchange = pd.read_parquet(HISTORICAL_CURRENCY_EXCHANGE_PATH)
historical_currency_exchange.head(5)

,date,AUD,BRL,CAD,EUR,GBP,JPY,MXN,NGN,RUB,SGD,USD
0,2024-09-30,1.443654,5.434649,1.351196,0.895591,0.747153,142.573268,19.694724,1668.736400,94.133735,1.280156,1
1,2024-10-01,1.442917,5.444170,1.352168,0.897557,0.746956,143.831429,19.667561,1670.694524,92.898519,1.284352,1
2,2024-10-02,1.449505,5.425444,1.348063,0.903056,0.752241,143.806861,19.606748,1669.653006,94.583198,1.286983,1
3,2024-10-03,1.456279,5.442044,1.351451,0.906018,0.754584,146.916773,19.457701,1670.097873,95.655442,1.294391,1
4,2024-10-04,1.460930,5.477788,1.355260,0.906452,0.761891,146.592323,19.363467,1649.763738,94.755337,1.296800,1


## Data Quality Assessment and Preprocessing

In [5]:
def null_stats(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a DataFrame summarizing null vs non-null counts and percentages per column.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame to analyze.
    
    Returns
    -------
    pd.DataFrame
        Summary table indexed by column name with columns:
         - null_count
         - non_null_count
         - null_pct
         - non_null_pct
    """
    total = len(df)
    # Count nulls per column
    null_count = df.isnull().sum()
    # Compute non-nulls
    non_null_count = total - null_count
    # Build summary
    stats = pd.DataFrame({
        'null_count'     : null_count,
        'non_null_count' : non_null_count,
        'null_pct'       : (null_count / total * 100).round(2),
        'non_null_pct'   : (non_null_count / total * 100).round(2)
    })
    return stats

In [6]:
print(transaction_fraud_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7483766 entries, 0 to 7483765
Data columns (total 23 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   transaction_id           object        
 1   customer_id              object        
 2   card_number              int64         
 3   timestamp                datetime64[us]
 4   vendor_category          object        
 5   vendor_type              object        
 6   vendor                   object        
 7   amount                   float64       
 8   currency                 object        
 9   country                  object        
 10  city                     object        
 11  city_size                object        
 12  card_type                object        
 13  is_card_present          bool          
 14  device                   object        
 15  channel                  object        
 16  device_fingerprint       object        
 17  ip_address               ob

In [7]:
transaction_fraud_data_processed = transaction_fraud_data.copy()

In [8]:
"""
Проверяем содержание null значений в колонках датасета с транзакциями
"""
null_stats(transaction_fraud_data_processed)

,null_count,non_null_count,null_pct,non_null_pct
transaction_id,0,7483766,0.0,100.0
customer_id,0,7483766,0.0,100.0
card_number,0,7483766,0.0,100.0
timestamp,0,7483766,0.0,100.0
vendor_category,0,7483766,0.0,100.0
vendor_type,0,7483766,0.0,100.0
vendor,0,7483766,0.0,100.0
amount,0,7483766,0.0,100.0
currency,0,7483766,0.0,100.0
country,0,7483766,0.0,100.0


Датасет не имеет null значений в своих колонках.
<br>Что на самом деле редкая ситуация для real-world данных

In [9]:
"""
По описанию данных в README.md можно сделать вывод,
что поле transaction_id является уникальным ключ в датасте transaction_fraud_data.

Проверим, что у значений этой колонки нет дублей.
"""

dupe_txns = transaction_fraud_data_processed['transaction_id'].duplicated().sum()
dupe_txns


np.int64(6460)

Исследуем строки с одинаковым ```transaction_id```.

Ожидается, что строки с одинаковыми ```transaction_id``` будут совпадать и по оставшимся полям.
<br> Тогда можно рассматривать их как дубли, и убрать из данных

In [10]:
def count_equal_unequal_rows(df: pd.DataFrame, group_col: str, columns_to_drop: list[str] = []):
    """
    Return a DataFrame summarizing counts of equal (identical) and unequal rows 
    for each duplicated value in the specified group column.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame to analyze.
    
    group_col : str
        Name of the column to group by. The function will consider values in this column
        and identify duplicates, then compare the rest of the row contents within each group.

    columns_to_drop: list[str]
        Names of columns that should not be compared.

    Returns
    -------
    pd.DataFrame
        Summary table indexed by the duplicated values in `group_col`, with columns:
         - equal : int
             Number of repeated rows beyond the first occurrence (fully identical rows).
         - unequal : int
             Number of rows in the group that are not identical to any other row in that group.
    """
    # Only rows with duplicated values in group_col
    duplicated_df = df[df[group_col].duplicated(keep=False)]
    duplicated_df = duplicated_df.drop(labels=columns_to_drop, axis=1)

    # Apply inside each group
    def count_group(g):
        row_counts = g.value_counts()
        equal = sum(row_counts[row_counts > 1] - 1)
        unequal = len(g) - row_counts[row_counts > 1].sum()
        return pd.Series({'equal': equal, 'unequal': unequal})

    return duplicated_df.groupby(group_col).apply(count_group)

In [11]:
"""
Так поле last_hour_activity_json сотавное и имеет тип dict внури нашего data frame,
нам необходимо сделать его hashable копию, чтобы значения это колонки можно было сравнивать между собой.

Для этого сериализуем эту колонку в аналогичную, которая будет хранить json строки.
"""
transaction_fraud_data_processed['last_hour_activity_json'] = transaction_fraud_data_processed['last_hour_activity'].apply(lambda x: json.dumps(x, sort_keys=True))

In [12]:
row_stat = count_equal_unequal_rows(transaction_fraud_data_processed, 'transaction_id', ['last_hour_activity'])
row_stat

/var/folders/p2/skh823fd5t5fcl9_fbmxvw24rb6_10/T/ipykernel_64255/614493501.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return duplicated_df.groupby(group_col).apply(count_group)


,equal,unequal
transaction_id,,
TX_0003e9db,0,2
TX_0004e984,0,2
TX_0005b91b,0,2
TX_001a97b5,0,2
TX_0022820a,0,2
...,...,...
TX_ffc7fc2b,0,2
TX_ffcc6971,0,2
TX_ffdefa24,0,2


In [13]:
print(f'Кол-во различных transaction_id у которых нет разных строк: {len(row_stat[row_stat['unequal'] == 0])}')
print(f'Кол-во различных transaction_id у которых нет одинаковых строк: {len(row_stat[row_stat['equal'] == 0])}')
print(f'Набор значений для различных строк с одинаковым transaction_id:\n{list(row_stat['unequal'].unique())}')


Кол-во различных transaction_id у которых нет разных строк: 0
Кол-во различных transaction_id у которых нет одинаковых строк: 6453
Набор значений для различных строк с одинаковым transaction_id:
[np.int64(2), np.int64(3)]


In [14]:
# Рассмотри пример пары строк с одинаковым transaction_id
transaction_fraud_data_processed[transaction_fraud_data_processed['transaction_id'] == 'TX_0003e9db']

,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,country,...,device,channel,device_fingerprint,ip_address,is_outside_home_country,is_high_risk_vendor,is_weekend,last_hour_activity,is_fraud,last_hour_activity_json
101374,TX_0003e9db,CUST_33598,5569939078731834,2024-09-30 11:21:11.618160,Retail,online,Etsy,1038.06,SGD,Singapore,...,Safari,web,4aa8dcbec2113fb024c9a68927bdd7fa,61.228.32.65,False,False,False,"{'num_transactions': 206, 'total_amount': 4965...",False,"{""max_single_amount"": 833420.6414852374, ""num_..."
1327891,TX_0003e9db,CUST_19259,375459469913771,2024-10-05 12:49:42.066906,Gas,major,Mobil,52909.46,JPY,Japan,...,Android App,mobile,f8f54159fd38e31b77e50c847fa7f3c5,245.196.10.182,False,False,True,"{'num_transactions': 1490, 'total_amount': 905...",False,"{""max_single_amount"": 3835974.1291076676, ""num..."


Мы видим, что у всех строк с повторяющимися ```transaction_id``` отличаются оставшиеся колонки.
<br>Это значит, что мы не можем брать поле ```transaction_id``` за уникальный идентификатор.

Добавим новую колонку, которая сможем служить уникальным идентификатором.
<br> Попробуем объеденить ```transaction_id``` и ```device_fingerprint```

In [28]:
transaction_fraud_data_processed['custom_transaction_id'] = transaction_fraud_data['transaction_id'] + '_' + transaction_fraud_data['device_fingerprint']
col = transaction_fraud_data_processed.pop('custom_transaction_id')
transaction_fraud_data_processed.insert(0, 'custom_transaction_id', col)
transaction_fraud_data_processed.head()

,custom_transaction_id,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,...,device,channel,device_fingerprint,ip_address,is_outside_home_country,is_high_risk_vendor,is_weekend,last_hour_activity,is_fraud,last_hour_activity_json
0,TX_a0ad2a2a_e8e6160445c935fd0001501e4cbac8bc,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820,Restaurant,fast_food,Taco Bell,294.87,GBP,...,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,False,False,False,"{'num_transactions': 1197, 'total_amount': 334...",False,"{""max_single_amount"": 1925480.6324148502, ""num..."
1,TX_3599c101_a73043a57091e775af37f252b3a32af9,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464,Entertainment,gaming,Steam,3368.97,BRL,...,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,True,True,False,"{'num_transactions': 509, 'total_amount': 2011...",True,"{""max_single_amount"": 5149117.011434267, ""num_..."
2,TX_a9461c6d_218864e94ceaa41577d216b149722261,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762,Grocery,physical,Whole Foods,102582.38,JPY,...,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,False,False,False,"{'num_transactions': 332, 'total_amount': 3916...",False,"{""max_single_amount"": 1852242.1831665323, ""num..."
3,TX_7be21fc4_70423fa3a1e74d01203cf93b51b9631d,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466,Gas,major,Exxon,630.60,AUD,...,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,False,False,False,"{'num_transactions': 764, 'total_amount': 2201...",False,"{""max_single_amount"": 2055798.460682913, ""num_..."
4,TX_150f490b_9880776c7b6038f2af86bd4e18a1b1a4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063,Healthcare,medical,Medical Center,724949.27,NGN,...,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,True,False,False,"{'num_transactions': 218, 'total_amount': 4827...",True,"{""max_single_amount"": 1157231.252130005, ""num_..."


In [ ]:
# проверим, что новое поле не содержит дубликатов
dupe_custom_txns = transaction_fraud_data_processed['custom_transaction_id'].duplicated().sum()
dupe_custom_txns

np.int64(0)

### Проверка категориальных полей

In [34]:
def get_uniq_category_field_values(df: pd.DataFrame, categ_columns: list[str]):
    for cc in categ_columns:
        print(f"Уникальные значения колонки {cc}:")
        print(df[cc].unique().tolist())
        print()

In [37]:
print("Колонки, относящиеся к информации о вендоре:\n")
vendor_categ_columns = [
    "vendor_category",
    "vendor_type",
    "vendor"
]
get_uniq_category_field_values(transaction_fraud_data_processed, categ_columns)

Колонки, относящиеся к информации о вендоре:

Уникальные значения колонки vendor_category:
['Restaurant', 'Entertainment', 'Grocery', 'Gas', 'Healthcare', 'Education', 'Travel', 'Retail']

Уникальные значения колонки vendor_type:
['fast_food', 'gaming', 'physical', 'major', 'medical', 'online', 'hotels', 'pharmacy', 'premium', 'events', 'supplies', 'airlines', 'local', 'booking', 'streaming', 'transport', 'casual']

Уникальные значения колонки vendor:
['Taco Bell', 'Steam', 'Whole Foods', 'Exxon', 'Medical Center', 'Coursera', 'Instacart', 'Westin', 'eBay', 'DuaneReade', 'Nobu', 'FreshDirect', 'StubHub', 'University Bookstore', 'American Airlines', 'Home Depot', "Morton's", 'Ticketmaster', 'Skillshare', 'Local Gas Station', 'Texaco', 'Hotels.com', 'Walmart Grocery', 'Epic Games', 'AMC Theaters', 'Etsy', 'IKEA', 'Costco', 'Spotify', 'Capital Grille', "Trader Joe's", 'Urgent Care', 'Careem', "Wendy's", "Macy's", 'Udemy', 'Target', 'Truck Stop', 'BP', 'TGI Fridays', 'Chevron', 'Amazon', '

In [42]:
print("Валюта:\n")
currency_categ_columns = ["currency"]
get_uniq_category_field_values(transaction_fraud_data_processed, currency_categ_columns)
print()
print('Валюты с известынм курсом:')
print(historical_currency_exchange.columns.to_list()[1:])

Валюта:

Уникальные значения колонки currency:
['GBP', 'BRL', 'JPY', 'AUD', 'NGN', 'EUR', 'MXN', 'RUB', 'CAD', 'SGD', 'USD']


Валюты с известынм курсом:
['AUD', 'BRL', 'CAD', 'EUR', 'GBP', 'JPY', 'MXN', 'NGN', 'RUB', 'SGD', 'USD']


In [44]:
print("Проверка гео фичей:\n")
geo_categ_columns = ['country', 'city', 'city_size']
get_uniq_category_field_values(transaction_fraud_data_processed, geo_categ_columns)

Проверка гео фичей:

Уникальные значения колонки country:
['UK', 'Brazil', 'Japan', 'Australia', 'Nigeria', 'Germany', 'Mexico', 'Russia', 'France', 'Canada', 'Singapore', 'USA']

Уникальные значения колонки city:
['Unknown City', 'San Antonio', 'Philadelphia', 'Phoenix', 'San Diego', 'Los Angeles', 'Chicago', 'Dallas', 'New York', 'San Jose', 'Houston']

Уникальные значения колонки city_size:
['medium', 'large']



Можно заметить, что определены только некоторые города из USA.

In [47]:
def check_ip_version(ip):
    try:
        ip_obj = ipaddress.ip_address(ip)
        return 'IPv4' if isinstance(ip_obj, ipaddress.IPv4Address) else 'IPv6'
    except ValueError:
        return 'Invalid'

In [48]:
# Проверим корректность указанных IP адресов
ip_versions = transaction_fraud_data_processed['ip_address'].apply(check_ip_version)
ip_versions.unique().tolist()

['IPv4']

In [50]:
# Проверим содержание фродовых транзакций в датасете.

total_tr = len(transaction_fraud_data_processed)
fraud_tr = len(transaction_fraud_data_processed[transaction_fraud_data_processed['is_fraud'] == 1])

print(f'Кол-во различных транзакций всего: {total_tr}')
print(f'Кол-во фродовых транзакций: {fraud_tr}')
print(f'% фродовых транзакций: {round(fraud_tr / total_tr, 4) * 100}%')




Кол-во различных транзакций всего: 7483766
Кол-во фродовых транзакций: 1494719
% фродовых транзакций: 19.97%


Можно заметить, что почти 1/5 всех транзакций - фродовые.
<br> Поэтому мы **не можем** назвать наш датасет highly unbalanced

### Проверка количественных полей

In [52]:
print(f'Самая старая транзакция:{transaction_fraud_data_processed['timestamp'].min()}')
print(f'Самая новая транзакция:{transaction_fraud_data_processed['timestamp'].max()}')


Самая старая транзакция:2024-09-30 00:00:01.034820
Самая новая транзакция:2024-10-30 23:59:59.101885


Период охватывает один календарный месяц и выглядит реалистично.
<br> В тоже время, это ограничивает возможность анализа сезонных колебаний в разрезе времён года или лет.

In [54]:
def convert_to_usd(row):
    currency = row['currency']
    amount = row['amount']
    
    # If already in USD, no conversion needed
    if currency == 'USD':
        return amount
    
    # Get the exchange rate for the currency
    # If exchange rate is how many units of currency equal 1 USD
    if currency in row and pd.notna(row[currency]):
        return amount / row[currency]
    else:
        # Handle missing exchange rates
        return None

In [56]:
# Проверка диапазона суммы транзакции

# Для начала конфертируем все суммы в доллары:
transaction_fraud_data_processed['date'] = transaction_fraud_data_processed['timestamp'].dt.date

merged_data = pd.merge(
    transaction_fraud_data_processed, 
    historical_currency_exchange, 
    on='date', 
    how='left'
)
merged_data['amount_usd'] = merged_data.apply(convert_to_usd, axis=1)
merged_data.columns

Index(['custom_transaction_id', 'transaction_id', 'customer_id', 'card_number',
       'timestamp', 'vendor_category', 'vendor_type', 'vendor', 'amount',
       'currency', 'country', 'city', 'city_size', 'card_type',
       'is_card_present', 'device', 'channel', 'device_fingerprint',
       'ip_address', 'is_outside_home_country', 'is_high_risk_vendor',
       'is_weekend', 'last_hour_activity', 'is_fraud',
       'last_hour_activity_json', 'date', 'AUD', 'BRL', 'CAD', 'EUR', 'GBP',
       'JPY', 'MXN', 'NGN', 'RUB', 'SGD', 'USD', 'amount_usd'],
      dtype='object')

In [57]:
transaction_fraud_data_processed = merged_data[transaction_fraud_data_processed.columns.to_list() + ['amount_usd']].copy()
transaction_fraud_data_processed.columns


Index(['custom_transaction_id', 'transaction_id', 'customer_id', 'card_number',
       'timestamp', 'vendor_category', 'vendor_type', 'vendor', 'amount',
       'currency', 'country', 'city', 'city_size', 'card_type',
       'is_card_present', 'device', 'channel', 'device_fingerprint',
       'ip_address', 'is_outside_home_country', 'is_high_risk_vendor',
       'is_weekend', 'last_hour_activity', 'is_fraud',
       'last_hour_activity_json', 'date', 'amount_usd'],
      dtype='object')

In [58]:
print(f'Минимальная сумма транзакции: {round(transaction_fraud_data_processed['amount_usd'].min(), 2)}$')
print(f'Максимальная сумма транзакции: {round(transaction_fraud_data_processed['amount_usd'].max(), 2)}$')

Минимальная сумма транзакции: 0.0$
Максимальная сумма транзакции: 15781.54$


Диапазон суммы транзакции выглядит реалистично